# Diagnóstico e monitoração da via férrea através de vagão instrumentado

Textinho

# Importação de Bibliotecas

In [4]:
from copy import deepcopy
import os
import pickle
import sys

from tqdm import tqdm
import numpy as np
import scipy.io as io
import pandas as pd
import h5py
import pyarrow

# Diretório

In [16]:
# Diretório do Pc do Jordan
path = 'H:\\.shortcut-targets-by-id\\1oKn7IN7zznQuhwjDCDdjq8r9wHJYBEhj\\DR_Train\\LRV4313'

print(f'Começando o processamento do vagão referente aos arquivos em: {path}...')
acc_path = f'{path}\\accelerometer_data\\'
gps_path = f'{path}\\gps_data\\'

acc_names = os.listdir(acc_path)
gps_names = os.listdir(gps_path)

Começando o processamento do vagão referente aos arquivos em: H:\.shortcut-targets-by-id\1oKn7IN7zznQuhwjDCDdjq8r9wHJYBEhj\DR_Train\LRV4313...


# Dados de acelerômetros

Carregando dados dos acelerômetros

In [17]:
# Carregamento dos dados dos acelerômetros

print('Carregando os dados dos acelerômetros...')
data = {}
for file in tqdm(acc_names):
    splited_name = file.split('_')
    date = splited_name[1]
    daily_passing = splited_name[3]
    region_number = splited_name[4]
    running_direction = splited_name[5]
    sensor = splited_name[-1][0]

    if region_number == '5':
        try:    
            with h5py.File(f'{acc_path}/{file}', 'r') as f:
                sensor_data = f.get('save_var')
                sensor_data = np.array(sensor_data)  
        except:
            sensor_data = io.loadmat(f'{acc_path}/{file}')
            sensor_data = sensor_data['save_var'] 

        sensor_data = sensor_data.reshape((-1,))
        sensor_data = sensor_data.astype('float32')

        if not data.get(f'{date}_{daily_passing}_{running_direction}'):
                data[f'{date}_{daily_passing}_{running_direction}'] = {}
                
        data[f'{date}_{daily_passing}_{running_direction}'][f'sensor_{sensor}'] = sensor_data


Carregando os dados dos acelerômetros...


 19%|█▉        | 1967/10159 [00:53<03:43, 36.62it/s] 


KeyboardInterrupt: 

Concatenação dos dados dos acelerômetros

In [15]:
print('Concatenação dos dados...')
full_df = pd.DataFrame()
for key, values in tqdm(data.items()):
    info = key.split('_')
    partial_df = pd.DataFrame()
    for k in values.keys():
        partial_df[k] = values[k]

    partial_df['date'] = int(info[0])
    partial_df['daily_passing'] = int(info[1])
    partial_df['running_direction'] = int(info[2])
    
    if full_df.empty:
        full_df = partial_df
    else:
        full_df = pd.concat([full_df, partial_df])

full_df.reset_index(inplace=True, drop=True)

Concatenação dos dados...


  2%|▏         | 7/311 [00:01<01:26,  3.50it/s]


ValueError: Length of values (695331) does not match length of index (716710)

In [ ]:
non_nulls = full_df.isnull().sum()
nulls = non_nulls.sum()
print(f'Número de dados nulos: {nulls}')


# Gravação dos dados de acelerômetro em .parquet

print('Salvando os dados em .parquet...')

os.makedirs('accel_data/', exist_ok=True)

prefix = path.split('/')[-1]
full_df.to_parquet(f'accel_data/{prefix}_acc.parquet', index=False)

# Dados de GPS

Carregamento dos dados dos GPS

In [18]:
data_gps = {}
print('Carregando os dados dos GPS...')
for file in tqdm(gps_names):
    if file[-4:] == '.ini':
        pass
    else:
        splited_name = file.split('_')
        date = splited_name[1]
        daily_passing = splited_name[3]
        region_number = splited_name[4]
        running_direction = splited_name[-1][0]

        if region_number == '5':
            gps_data = io.loadmat(f'{gps_path}/{file}')
            gps_data = gps_data['save_var_gps']

            if not data_gps.get(f'{date}_{daily_passing}_{running_direction}'):
                data_gps[f'{date}_{daily_passing}_{running_direction}'] = {}

            for i in range(gps_data.shape[1]):
                data_gps[f'{date}_{daily_passing}_{running_direction}'][f'gps_{i}'] = gps_data[:, i]

Carregando os dados dos GPS...


100%|██████████| 1460/1460 [00:06<00:00, 228.47it/s]


Concatenação dos dados de gps

In [74]:
print('Concatenação dos dados...')
full_df = pd.DataFrame()
for key, values in tqdm(data_gps.items()):
    info = key.split('_')
    partial_df = pd.DataFrame()
    for k in values.keys():
        partial_df[k] = values[k]

    partial_df['date'] = int(info[0])
    partial_df['daily_passing'] = int(info[1])
    partial_df['running_direction'] = int(info[2])
    
    if full_df.empty:
        full_df = partial_df
    else:
        full_df = pd.concat([full_df, partial_df])

full_df.reset_index(inplace=True, drop=True)

Concatenação dos dados...


100%|██████████| 295/295 [00:02<00:00, 120.23it/s]


In [83]:
non_nulls = full_df.isnull().sum()
nulls = non_nulls.sum()
print(f'Número de dados nulos: {nulls}')

# Gravação dos dados de GPS em .parquet

print('Salvando os dados em .parquet...')

os.makedirs('gps_data/', exist_ok=True)

prefix = path.split('\\')[-1]
full_df.to_parquet(f'gps_data/{prefix}_gps.parquet', index=False)

Número de dados nulos: 0
Salvando os dados em .parquet...
